### Construir un modelo de clasificación para predecir la probabilidad de diabetes en pacientes usando  el  conjunto  de  datos  Pima  Indians  Diabetes  Dataset,  aplicando  Grid  Search  y  Random  Search para optimizar el rendimiento del modelo y comparar sus resultados en términos de precisión, F1-score y eficiencia computacional.
----

## 1\. Carga y Exploración de Datos 📊

Primero, cargamos el conjunto de datos de diabetes de Pima Indians desde la URL https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv. Usamos la librería **pandas** para leer el archivo CSV. Es crucial asignar nombres a las columnas, ya que el archivo original no los incluye. Luego, exploramos los datos para entender su estructura, los tipos de cada variable y sus estadísticas descriptivas básicas.

#### Descripción de las Variables:

* **preg:** Número de embarazos.
* **plas:** Concentración de glucosa en plasma a las 2 horas en una prueba de tolerancia a la glucosa oral. Un valor alto es un indicador clave de diabetes.
* **pres:** Presión arterial diastólica (mm Hg). La hipertensión a menudo se asocia con la diabetes.
* **skin:** Grosor del pliegue cutáneo del tríceps (mm). Puede indicar la adiposidad corporal.
* **test:** Insulina sérica de 2 horas (mu U/ml). Niveles anormales de insulina están directamente relacionados con la diabetes.
* **mass:** Índice de Masa Corporal (IMC). La obesidad es un factor de riesgo importante para la diabetes tipo 2.
* **pedi:** Función de pedigrí de diabetes. Proporciona una medida de la predisposición genética a la diabetes.
* **age:** Edad (años). El riesgo de diabetes tipo 2 aumenta con la edad.
* **class:** Variable de clase (0 o 1). Indica si el paciente tiene diabetes (1) o no (0). Esta es nuestra variable objetivo.

<!-- end list -->

In [3]:
# Importación de la librería para manipulación de datos
import pandas as pd

# 1. CARGA Y EXPLORACIÓN DE DATOS

# URL del conjunto de datos
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
# Nombres de las columnas según la descripción del dataset
column_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
# Cargar los datos en un DataFrame de pandas
data = pd.read_csv(url, header=None, names=column_names)

# Mostrar las primeras 5 filas para una vista rápida
print("### 1. Carga y Exploración de Datos ###")
print("\nPrimeras 5 filas del conjunto de datos:")
print(data.head())

# Mostrar información general (tipos de datos, valores no nulos)
print("\nInformación del conjunto de datos:")
data.info()

# Mostrar estadísticas descriptivas (media, desviación estándar, etc.)
print("\nEstadísticas descriptivas:")
print(data.describe())

### 1. Carga y Exploración de Datos ###

Primeras 5 filas del conjunto de datos:
   preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
2     8   183    64     0     0  23.3  0.672   32      1
3     1    89    66    23    94  28.1  0.167   21      0
4     0   137    40    35   168  43.1  2.288   33      1

Información del conjunto de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   preg    768 non-null    int64  
 1   plas    768 non-null    int64  
 2   pres    768 non-null    int64  
 3   skin    768 non-null    int64  
 4   test    768 non-null    int64  
 5   mass    768 non-null    float64
 6   pedi    768 non-null    float64
 7   age     768 non-null    int64  
 8   class   768 non-null    int64  
dtypes: float64(2), int64(7)
m

-----

## 2\. Preprocesamiento de Datos ⚙️

Antes de entrenar el modelo, preparamos los datos.

  * **Separación de Variables:** Dividimos el dataset en características (`X`) y la variable objetivo (`y`).
  * **División de Datos:** Particionamos los datos en un conjunto de entrenamiento (70%) y uno de prueba (30%). Usamos `random_state` para que esta división sea siempre la misma y nuestros resultados sean reproducibles.
  * **Escalado de Variables:** Aplicamos `StandardScaler` para estandarizar las características numéricas. Esto es importante porque los algoritmos de Machine Learning funcionan mejor cuando las variables tienen una escala similar. Se ajusta el escalador **solo** con los datos de entrenamiento (`fit_transform`) y luego se aplica esa misma transformación a los datos de prueba (`transform`) para evitar la fuga de datos.

<!-- end list -->

In [4]:
# Importación de las herramientas de preprocesamiento y modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 2. PREPROCESAMIENTO

# Separar las características (X) y la variable objetivo (y)
X = data.drop('class', axis=1)  # Todas las columnas excepto la clase
y = data['class']              # Solo la columna de la clase

# Dividir los datos: 70% para entrenamiento, 30% para prueba
# random_state asegura que la división sea reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Inicializar el escalador estándar
scaler = StandardScaler()
# Ajustar el escalador en los datos de entrenamiento y transformar ambos conjuntos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n### 2. Preprocesamiento de Datos ###")
print(f"Forma del conjunto de entrenamiento (X_train): {X_train_scaled.shape}")
print(f"Forma del conjunto de prueba (X_test): {X_test_scaled.shape}")


### 2. Preprocesamiento de Datos ###
Forma del conjunto de entrenamiento (X_train): (537, 8)
Forma del conjunto de prueba (X_test): (231, 8)


-----

## 3\. Modelo Base 🎯

Creamos y evaluamos un modelo `RandomForestClassifier` sin ningún tipo de optimización, usando sus hiperparámetros por defecto. Esto nos sirve como un punto de referencia (baseline) para comparar los resultados de los modelos optimizados. Lo evaluamos con métricas clave:

  * **Accuracy:** Porcentaje de predicciones correctas.
  * **Recall (Sensibilidad):** Capacidad del modelo para encontrar todos los casos positivos.
  * **F1-Score:** Media armónica de precisión y recall, útil en clases desbalanceadas.
  * **AUC (Área bajo la curva ROC):** Mide la capacidad del modelo para distinguir entre clases.

<!-- end list -->

In [5]:
# Importación del modelo y las métricas de evaluación
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report
import time

# 3. MODELO BASE

# Inicializar el clasificador con parámetros por defecto
base_model = RandomForestClassifier(random_state=42)

# Medir el tiempo de entrenamiento
start_time_base = time.time()
# Entrenar el modelo con los datos de entrenamiento escalados
base_model.fit(X_train_scaled, y_train)
# Registrar el tiempo de finalización del entrenamiento
end_time_base = time.time()
# Calcular el tiempo total de entrenamiento
training_time_base = end_time_base - start_time_base

# Realizar predicciones en el conjunto de prueba
y_pred_base = base_model.predict(X_test_scaled)
# Predecir las probabilidades para calcular el AUC
y_prob_base = base_model.predict_proba(X_test_scaled)[:, 1]

# Calcular las métricas de evaluación /  Evaluar el rendimiento del modelo base
accuracy_base = accuracy_score(y_test, y_pred_base)
recall_base = recall_score(y_test, y_pred_base)
f1_base = f1_score(y_test, y_pred_base)
auc_base = roc_auc_score(y_test, y_prob_base)

# Imprimir las métricas de evaluación del modelo base
print("\n### 3. Resultados del Modelo Base ###")
print(f"Accuracy: {accuracy_base:.4f}")
print(f"Recall: {recall_base:.4f}")
print(f"F1-Score: {f1_base:.4f}")
print(f"AUC: {auc_base:.4f}")
print(f"Tiempo de entrenamiento: {training_time_base:.4f} segundos")
print("\nReporte de Clasificación del Modelo Base:")
print(classification_report(y_test, y_pred_base))


### 3. Resultados del Modelo Base ###
Accuracy: 0.7576
Recall: 0.6625
F1-Score: 0.6543
AUC: 0.8046
Tiempo de entrenamiento: 0.2123 segundos

Reporte de Clasificación del Modelo Base:
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       151
           1       0.65      0.66      0.65        80

    accuracy                           0.76       231
   macro avg       0.73      0.74      0.73       231
weighted avg       0.76      0.76      0.76       231



-----

## 4\. Aplicar Grid Search

**Grid Search** es una técnica de optimización que prueba de manera exhaustiva todas las combinaciones posibles de un conjunto de hiperparámetros que le proporcionamos. Definimos una "cuadrícula" (`param_grid`) con los valores que queremos probar para `n_estimators`, `max_depth` y `min_samples_split`. `GridSearchCV` entrena un modelo para cada combinación usando validación cruzada (`cv=5`) y selecciona la que ofrece el mejor rendimiento.

In [6]:
# Importación de GridSearchCV
from sklearn.model_selection import GridSearchCV

# 4. APLICAR GRID SEARCH

# Definir la cuadrícula de hiperparámetros a probar
param_grid = {
    'n_estimators': [50, 100, 200],      # Número de árboles
    'max_depth': [None, 10, 20],         # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10]    # Muestras mínimas para dividir un nodo
}

# Inicializar GridSearchCV
# n_jobs=-1 usa todos los núcleos de CPU para acelerar la búsqueda
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

# Medir el tiempo de la búsqueda
start_time_grid = time.time()
# Ejecutar la búsqueda en los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)
# Registrar el tiempo de finalización de la búsqueda
end_time_grid = time.time()
# Calcular el tiempo total de la búsqueda
training_time_grid = end_time_grid - start_time_grid

# Obtener los mejores hiperparámetros encontrados
best_params_grid = grid_search.best_params_
# Obtener el mejor modelo encontrado por Grid Search
best_model_grid = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba con el mejor modelo
y_pred_grid = best_model_grid.predict(X_test_scaled)
# Predecir las probabilidades para calcular el AUC
y_prob_grid = best_model_grid.predict_proba(X_test_scaled)[:, 1]

# Evaluar el rendimiento del modelo optimizado con Grid Search
accuracy_grid = accuracy_score(y_test, y_pred_grid)
recall_grid = recall_score(y_test, y_pred_grid)
f1_grid = f1_score(y_test, y_pred_grid)
auc_grid = roc_auc_score(y_test, y_prob_grid)

# Imprimir los resultados
print("\n### 4. Resultados de Grid Search ###")
print(f"Mejores Hiperparámetros: {best_params_grid}")
print(f"Accuracy: {accuracy_grid:.4f}")
print(f"F1-Score: {f1_grid:.4f}")
print(f"AUC: {auc_grid:.4f}")
print(f"Tiempo de búsqueda: {training_time_grid:.4f} segundos")
print("\nReporte de Clasificación (Grid Search):")
print(classification_report(y_test, y_pred_grid))


### 4. Resultados de Grid Search ###
Mejores Hiperparámetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.7576
F1-Score: 0.6500
AUC: 0.8046
Tiempo de búsqueda: 31.7483 segundos

Reporte de Clasificación (Grid Search):
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       151
           1       0.65      0.65      0.65        80

    accuracy                           0.76       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.76      0.76       231



-----

## 5\. Aplicar Random Search 🎲

A diferencia de Grid Search, **Random Search** no prueba todas las combinaciones, sino que selecciona un número fijo de combinaciones de hiperparámetros (`n_iter`) de manera aleatoria a partir de un rango o distribución de valores. Esto la hace mucho más rápida y eficiente, especialmente cuando el espacio de búsqueda es grande. A menudo, encuentra modelos muy buenos (o incluso los mejores) en mucho menos tiempo.

In [10]:
# Importación de RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

# 5. APLICAR RANDOM SEARCH

# Definir una distribución de hiperparámetros (puede ser más amplia)
param_dist = {
    'n_estimators': range(50, 301, 50),
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15]
}

# Inicializar RandomizedSearchCV
# n_iter=10 significa que probará 10 combinaciones aleatorias
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                                   param_distributions=param_dist,
                                   n_iter=10,  # Número de combinaciones de parámetros a probar
                                   cv=5,
                                   scoring='accuracy',
                                   n_jobs=-1,
                                   random_state=42)

# Registrar el tiempo de inicio de la búsqueda
start_time_random = time.time()
# Ejecutar la búsqueda aleatoria en los datos de entrenamiento
random_search.fit(X_train_scaled, y_train)
# Registrar el tiempo de finalización de la búsqueda
end_time_random = time.time()
# Calcular el tiempo total de la búsqueda
training_time_random = end_time_random - start_time_random

# Obtener los mejores hiperparámetros encontrados
best_params_random = random_search.best_params_
# Obtener el mejor modelo encontrado por Random Search
best_model_random = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba con el mejor modelo
y_pred_random = best_model_random.predict(X_test_scaled)
# Predecir las probabilidades para calcular el AUC
y_prob_random = best_model_random.predict_proba(X_test_scaled)[:, 1]

# Evaluar el rendimiento del modelo optimizado con Random Search
accuracy_random = accuracy_score(y_test, y_pred_random)
recall_random = recall_score(y_test, y_pred_random)
f1_random = f1_score(y_test, y_pred_random)
auc_random = roc_auc_score(y_test, y_prob_random)

# Imprimir los resultados
print("\n### 5. Resultados de Random Search ###")
print(f"Mejores Hiperparámetros: {best_params_random}")
print(f"Accuracy: {accuracy_random:.4f}")
print(f"F1-Score: {f1_random:.4f}")
print(f"AUC: {auc_random:.4f}")
print(f"Tiempo de búsqueda: {training_time_random:.4f} segundos")
print("\nReporte de Clasificación (Random Search):")
print(classification_report(y_test, y_pred_random))


### 5. Resultados de Random Search ###
Mejores Hiperparámetros: {'n_estimators': 250, 'min_samples_split': 2, 'max_depth': None}
Accuracy: 0.7576
F1-Score: 0.6500
AUC: 0.8054
Tiempo de búsqueda: 24.9731 segundos

Reporte de Clasificación (Random Search):
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       151
           1       0.65      0.65      0.65        80

    accuracy                           0.76       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.76      0.76       231



-----

## 6\. Comparación y Análisis 📈

Para facilitar la comparación, consolidamos las métricas y los tiempos de entrenamiento de los tres modelos (Base, Grid Search y Random Search) en una tabla. Esto nos permite ver de un vistazo qué técnica ofreció el mejor equilibrio entre rendimiento y costo computacional.

In [11]:
# 6. COMPARACIÓN Y ANÁLISIS

# Crear un DataFrame para comparar los resultados de manera ordenada
comparison_data = {
    'Modelo': ['Base', 'Grid Search', 'Random Search'],
    'Accuracy': [accuracy_base, accuracy_grid, accuracy_random],
    'Recall': [recall_base, recall_grid, recall_random],
    'F1-Score': [f1_base, f1_grid, f1_random],
    'AUC': [auc_base, auc_grid, auc_random],
    'Tiempo (s)': [training_time_base, training_time_grid, training_time_random]
}
comparison_df = pd.DataFrame(comparison_data).round(4) # Redondear para mejor visualización

# Imprimir la tabla de comparación
print("\n### 6. Tabla de Comparación de Modelos ###")
print(comparison_df)


### 6. Tabla de Comparación de Modelos ###
          Modelo  Accuracy  Recall  F1-Score     AUC  Tiempo (s)
0           Base    0.7576  0.6625    0.6543  0.8046      0.2123
1    Grid Search    0.7576  0.6500    0.6500  0.8046     31.7483
2  Random Search    0.7576  0.6500    0.6500  0.8054     24.9731


-----
## 7\. Reflexión Final 🧠

#### **¿Cuál técnica fue más eficiente? ⚙️**

La eficiencia se mide en términos de costo computacional, es decir, el **tiempo de ejecución**.

* **Modelo Base:** Con un tiempo de solo **0.21 segundos**, fue, por un margen abrumador, el método más eficiente. Esto es lógico, ya que no realiza ninguna búsqueda de hiperparámetros; simplemente entrena el modelo una vez.
* **Random Search:** Tardó **24.97 segundos** en ejecutarse.
* **Grid Search:** Fue la técnica más lenta, con un tiempo de **31.75 segundos**.

**Conclusión:** Entre las dos técnicas de optimización, **Random Search fue más eficiente que Grid Search**. Logró un rendimiento prácticamente idéntico al de Grid Search pero en aproximadamente un 20% menos de tiempo, ya que evalúa un subconjunto aleatorio de las combinaciones en lugar de probarlas todas exhaustivamente. Sin embargo, ninguna técnica de optimización se acercó a la eficiencia del modelo base.

---

#### **¿Cuál encontró el mejor modelo? 🏆**

Este es el punto más interesante y revelador del análisis. Basado en las métricas, el mejor modelo es, sorprendentemente, el **Modelo Base**.

* **Análisis de Métricas:**
    * **Accuracy:** Las tres técnicas arrojaron la misma exactitud (`0.7576`).
    * **AUC:** Random Search (`0.8054`) fue marginalmente superior al Modelo Base y Grid Search (`0.8046`). Sin embargo, una diferencia tan minúscula es, en la práctica, estadísticamente insignificante.
    * **F1-Score y Recall:** Aquí es donde el Modelo Base demuestra su superioridad. Obtuvo un **F1-Score de 0.6543** y un **Recall de 0.6625**, ambos superiores a los `0.6500` que lograron tanto Grid Search como Random Search.

**Conclusión:** A pesar del tiempo y el cómputo invertido, **ninguna de las técnicas de optimización logró encontrar una combinación de hiperparámetros que superara al modelo con su configuración por defecto**. Esto no significa que la optimización falló, sino que nos enseña algo crucial: los parámetros predeterminados del `RandomForestClassifier` de Scikit-learn ya son muy robustos y estaban excepcionalmente bien adaptados para este problema en particular. Por lo tanto, considerando el equilibrio entre rendimiento (F1-Score/Recall) y simplicidad, **el Modelo Base es el ganador**.

---

#### **¿Qué hubieras hecho diferente? 🤔**

El hecho de que el modelo base ganara es una oportunidad perfecta para refinar el proceso. Aquí hay varias cosas que se podrían haber hecho de manera diferente:

1.  **Revisar el Preprocesamiento de Datos:** Esta es la sospecha principal. El dataset de Pima Indians es conocido por tener valores "cero" en columnas donde es fisiológicamente imposible (ej. `plas`, `pres`, `mass`). Si estos ceros no se trataron como datos faltantes (por ejemplo, reemplazándolos con la media o la mediana de la columna), el modelo está aprendiendo de datos incorrectos. Un preprocesamiento más cuidadoso podría haber creado una base de datos donde la optimización sí encontrara mejoras significativas.

2.  **Refinar el Espacio de Búsqueda:** El primer intento de búsqueda de hiperparámetros es a menudo una exploración amplia. Dado que el modelo base funcionó tan bien, un segundo paso lógico sería realizar una búsqueda mucho más **fina y acotada alrededor de los parámetros por defecto**. Por ejemplo, si el `n_estimators` por defecto es 100, en lugar de probar `[50, 100, 200]`, se podría probar `[90, 100, 110, 120]` para buscar mejoras más sutiles.

3.  **Experimentar con Otros Algoritmos:** Quizás `RandomForest` ya alcanzó su máximo potencial en este dataset. Se podría haber experimentado con otros tipos de modelos que podrían ser más sensibles al ajuste de hiperparámetros, como **XGBoost, LightGBM, o incluso un Support Vector Machine (SVM)**, para ver si ofrecen un rendimiento superior.

4.  **Optimizar para una Métrica Específica:** La búsqueda se optimizó para `accuracy` (según el código previo) o una métrica general. En un problema médico como la detección de diabetes, minimizar los falsos negativos (pacientes con diabetes que no son detectados) es crítico. Por lo tanto, se podría haber configurado la búsqueda para optimizar específicamente el **Recall** (`scoring='recall'`), lo que podría haber llevado a un modelo diferente y más útil desde el punto de vista clínico, aunque su `accuracy` general fuera menor.